In [51]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import SentenceSplitter
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

import chromadb 
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
from llama_index.core import VectorStoreIndex
from llama_index.core import load_index_from_storage
from llama_index.core import Document

from sklearn.metrics.pairwise import cosine_similarity

from dotenv import load_dotenv, find_dotenv 
import os 
from llama_index.llms.groq import Groq 

import gradio as gr
import pandas as pd 
from pathlib import Path

In [52]:
df = pd.read_csv("aulas_todas_materias.csv")
documents = []

for _, row in df.iterrows():
    content = f"Aula: {row['data']}\n\n{row['texto']}" 
    metadata = {
        'data': row['data'],
        'bimestre': int(row['bimestre']),
        'materia': row['materia'].lower()
    }
    doc = Document(text=content, metadata=metadata)
    documents.append(doc)

documents



[Document(id_='45f5cbe9-a7bf-438f-80d9-f6e8228af0ff', embedding=None, metadata={'data': '2 ABR', 'bimestre': 2, 'materia': 'matematica'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='Aula: 2 ABR\n\n2 ABR\nOBJETIVOS DE APRENDIZAGEM \nDO DC-GOEM - (GO-\nEMMAT501A) Compreender o \nconceito de função polinomial do 1º\ngrau, identificando a relação entre \nduas variáveis apresentadas em\ntextos de origem socioeconômicas \ne/ou de natureza técnico ou\ncientífica, entre outros para resolver \nsituações problemas do cotidiano.\nOBJETOS DE CONHECIMENTO\nDO DC-GOEM - Funções\npolinomiais do 1º\ngrau (função afim,\nfunção linear,\nfunção constante,\nfunção identidade)\nAplicação de Atividades via \nrecursos digitais\nRealização de Atividades via recursos \ndigitais', path=None, url=None, mimetype=None), image_resource=None, audio_resource=

In [53]:
print(documents[0].get_metadata_str())

data: 2 ABR
bimestre: 2
materia: matematica


### Documentos em aprtes menores

In [54]:
node_parser = SentenceSplitter(chunk_size=1000)
nodes = node_parser.get_nodes_from_documents(documents, show_progress=True)

Parsing nodes: 100%|██████████| 295/295 [00:00<00:00, 3120.64it/s]


In [55]:
nodes

[TextNode(id_='a01565e2-fbb7-45c1-b103-c2f86ac66017', embedding=None, metadata={'data': '2 ABR', 'bimestre': 2, 'materia': 'matematica'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='45f5cbe9-a7bf-438f-80d9-f6e8228af0ff', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'data': '2 ABR', 'bimestre': 2, 'materia': 'matematica'}, hash='264a7761860bd46e22a007c0ee7b883af27a571aea993b6e410e59fe922badd3')}, metadata_template='{key}: {value}', metadata_separator='\n', text='Aula: 2 ABR\n\n2 ABR\nOBJETIVOS DE APRENDIZAGEM \nDO DC-GOEM - (GO-\nEMMAT501A) Compreender o \nconceito de função polinomial do 1º\ngrau, identificando a relação entre \nduas variáveis apresentadas em\ntextos de origem socioeconômicas \ne/ou de natureza técnico ou\ncientífica, entre outros para resolver \nsituações problemas do cotidiano.\nOBJETOS DE CONHECIMENTO\nDO DC-GOEM - Funções\npolinomiais do 1º\ngrau (função afim,\nfunção lin

### Gerando Embeddings

In [56]:
class ChromaEmbeddingWrapper:
    def __init__(self, model_name):
        self.model = HuggingFaceEmbedding(model_name=model_name)
        self.name = model_name
    
    def __call__(self, texts):
        return self.model.encode(texts, show_progress_bar=False).tolist()

In [57]:
current_file = Path().resolve()
model_folder = current_file / "all-MiniLM-L6-v2"
str(model_folder)

'D:\\Documentos\\tcc - IA\\projeto\\all-MiniLM-L6-v2'

In [58]:
chroma_client = chromadb.Client()

In [59]:
try:
    chroma_collection = chroma_client.get_or_create_collection("doccuments_llm")
except Exception as e:
    print(f"Erro ao carregar ou criar embeddings {e}")

### Salvando Embeddings no DB

In [60]:
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [61]:
embed_model = HuggingFaceEmbedding(model_name=str(model_folder))

In [62]:
index = VectorStoreIndex(nodes, storage_context=storage_context, embed_model=embed_model)

### Verificar conteúdos semelhantes entre bimestres

In [86]:
def get_text_for_bimestre(bimestre, materia, query_engine):
    prompt = f"Conteúdo do {bimestre}º bimestre de {materia}"
    response = query_engine.query(prompt)
    if hasattr(response, "response"):
        return response.response
    elif isinstance(response, str):
        return response
    else:
        return None
    
def get_embedding(text, embed_model):
    return embed_model([text])[0]


def check_revision_needed(bimestre_atual, materia, query_engine, embed_model, threshold=0.7):
    if bimestre_atual <= 1:
        return False
    
    texto_atual = get_text_for_bimestre(bimestre_atual, materia, query_engine)
    texto_anterior = get_text_for_bimestre(bimestre_atual - 1, materia, query_engine)

    if not texto_atual or not texto_anterior:
        return False
    
    emb_atual = get_embedding(texto_atual, embed_model)  # usa a função com embed_documents
    emb_ant = get_embedding(texto_anterior, embed_model)
    
    sim = cosine_similarity([emb_atual], [emb_ant])[0][0]
    print(f"Similaridade entre bimestre {bimestre_atual} e {bimestre_atual - 1} para {materia}: {sim:.4f}")
    
    return sim >= threshold


### Recuperação de informações

In [64]:
load_dotenv(find_dotenv())

True

In [65]:
GROQ_API = os.environ.get("GROQ_API")

In [66]:
llm = Groq(
    model="llama3-70b-8192",
    api_key=GROQ_API
)

In [67]:
query_engine = index.as_query_engine(llm=llm, similarity_top_k=2)

#### Testando o modelo

In [94]:
aluno = "João da Silva"
nota = 4.5
bimestre = 1
materia = "matematica"

def gerar_plano_estudo(aluno, nota, bimestre, materia, query_engine):
    if nota >= 8:
        return f"O aluno {aluno} teve nota {nota} no {bimestre}º bimestre. Nenhum plano de estudo é necessário."

    #revisao = check_revision_needed(bimestre, materia, query_engine, embed_model)

    #revisao_texto = ""
    #if revisao:
    #    revisao_texto = "Observe que o conteúdo deste bimestre está relacionado ao bimestre anterior. Recomenda-se revisar também os conteúdos do bimestre anterior.\n\n"


    prompt = f"""
    O aluno {aluno} obteve nota {nota} no {bimestre}º bimestre na mataria de {materia}.
    
    Por favor, gere um plano de estudo detalhado **APENAS PARA A MATÉRIA {materia.upper()}**, ignorando informações de outras matérias.

    Nota abaixo de 6 é ruim entre 6 e 7 mediana, acima de 7 boa
    Com base no conteúdo das aulas deste bimestre, gere um plano de estudo com:
    - Conteúdos principais a revisar
    - Dicas de como estudar
    - Sugestão de atividades
    - Gere conteúdos complementares para auxiliar no contéudo prinpal
    Use linguagem simples e voltada para o aluno.
    """
    resp = query_engine.query(prompt)
    return resp

In [96]:
gerar_plano_estudo(aluno=aluno, nota=6.5, bimestre=3,materia="Matematica", query_engine=query_engine)

Response(response='Olá João!\n\nVamos criar um plano de estudo detalhado para você melhorar sua nota em Matemática.\n\n**Conteúdos principais a revisar:**\n\n1. Sistemas e unidades de medida: leitura e conversão de unidades de grandezas diversas.\n2. Interpretação de dados e informações que envolvam variação entre grandezas.\n3. Análise de gráficos (funções e/ou taxas de variação) para avaliar situações gerais.\n4. Teoria de conjuntos.\n\n**Dicas de como estudar:**\n\n* Leia atentamente as anotações das aulas e revise os conceitos principais.\n* Faça exercícios relacionados aos conteúdos mencionados acima.\n* Tente resolver problemas semelhantes aos que você encontrou nas aulas.\n* Use recursos visuais, como gráficos e diagramas, para ajudar a entender melhor os conceitos.\n\n**Sugestão de atividades:**\n\n* Faça uma lista de exercícios sobre sistemas e unidades de medida e resolva-os.\n* Analise gráficos de diferentes tipos (funções e taxas de variação) e tente interpretar os resultad

In [ ]:
import gradio as gr

def interface_plano_com_feedback(aluno, nota, bimestre, materia):
    plano = gerar_plano_estudo(aluno, nota, bimestre, materia, query_engine, index)
    return plano.response, gr.update(visible=True)

with gr.Blocks() as demo:
    aluno_input = gr.Textbox(label="Nome do aluno")
    nota_input = gr.Number(label="Nota do aluno")
    bimestre_input = gr.Number(label="Bimestre")
    materia_input = gr.Textbox(label="Matéria")
    
    btn_gerar = gr.Button("Gerar Plano de Estudo")
    saida_plano = gr.Textbox(label="Plano de Estudo")
    
    feedback_input = gr.Textbox(label="O plano estava adequado? Comentários")
    btn_feedback = gr.Button("Enviar Feedback")
    feedback_msg = gr.Textbox(visible=False)
    
    btn_gerar.click(interface_plano_com_feedback, inputs=[aluno_input, nota_input, bimestre_input, materia_input], outputs=saida_plano)
    #btn_feedback.click(salvar_feedback, inputs=[aluno_input, bimestre_input, materia_input, feedback_input], outputs=feedback_msg)

demo.launch() 


* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "d:\Documentos\tcc - IA\projeto\venv\lib\site-packages\gradio\queueing.py", line 626, in process_events
    response = await route_utils.call_process_api(
  File "d:\Documentos\tcc - IA\projeto\venv\lib\site-packages\gradio\route_utils.py", line 350, in call_process_api
    output = await app.get_blocks().process_api(
  File "d:\Documentos\tcc - IA\projeto\venv\lib\site-packages\gradio\blocks.py", line 2235, in process_api
    result = await self.call_function(
  File "d:\Documentos\tcc - IA\projeto\venv\lib\site-packages\gradio\blocks.py", line 1746, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
  File "d:\Documentos\tcc - IA\projeto\venv\lib\site-packages\anyio\to_thread.py", line 56, in run_sync
    return await get_async_backend().run_sync_in_worker_thread(
  File "d:\Documentos\tcc - IA\projeto\venv\lib\site-packages\anyio\_backends\_asyncio.py", line 2470, in run_sync_in_worker_thread
    return await f